# RAG Submissions Blender

Ноутбук для **ансамблирования (blending)** сабмитов по RAG.

Цель: из двух (или более) сабмитов получить один итоговый `submission.csv`,
используя простые, прозрачные правила без обучения нейросети.

Основной кейс — у тебя есть два файла:

- `sub1.csv` — сабмит №1 (например, baseline);
- `sub2.csv` — сабмит №2 (например, более тяжёлый или с другой моделью),

и нужно для каждого `id` выбрать лучший ответ или аккуратно их комбинировать.

## Block 0. Конфиг и импорты

In [ ]:
import os
import json
from pathlib import Path
from typing import List, Dict, Any

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import difflib
import re

BASE_DIR = Path('.').resolve()
INPUT_DIR = BASE_DIR / 'inputs'
OUTPUT_DIR = BASE_DIR / 'outputs'

INPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Пути к сабмитам (можно поменять под свою среду)
SUB1_PATH = INPUT_DIR / 'submission_1.csv'
SUB2_PATH = INPUT_DIR / 'submission_2.csv'

# (опционально) файл с ground truth для оффлайн-оценки
GT_PATH = INPUT_DIR / 'ground_truth.csv'  # можно оставить несуществующим

# Конфигурация колонок
ID_COL = 'id'
ANSWER_COL = 'answer'
REFS_COL = 'refs_json'  # если в сабмитах есть колонка с ссылками/чанками

# Пороговые значения для правил blending
MIN_ANSWER_LEN = 15   # ответы короче считаем подозрительно короткими
MAX_ANSWER_LEN = 800  # ответы длиннее считаем "простынями"
VERY_SIMILAR_THRESHOLD = 0.9  # похожесть текстов (SequenceMatcher)

print('BASE_DIR  :', BASE_DIR)
print('INPUT_DIR :', INPUT_DIR)
print('OUTPUT_DIR:', OUTPUT_DIR)


## Block 1. Загрузка и выравнивание сабмитов

In [ ]:
def load_submission(path: Path, name: str) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f'Не найден сабмит {name}: {path}')
    df = pd.read_csv(path)
    print(f'{name}: shape =', df.shape)
    if ID_COL not in df.columns or ANSWER_COL not in df.columns:
        raise KeyError(f'В сабмите {name} должны быть колонки {ID_COL!r} и {ANSWER_COL!r}')
    return df

sub1 = load_submission(SUB1_PATH, 'sub1')
sub2 = load_submission(SUB2_PATH, 'sub2')

# sanity-check по id
ids1 = set(sub1[ID_COL])
ids2 = set(sub2[ID_COL])
print('\nУникальных id в sub1:', len(ids1))
print('Уникальных id в sub2:', len(ids2))
print('Пересечение id:', len(ids1 & ids2))

if ids1 != ids2:
    print('\nWARNING: множества id отличаются. Будем брать пересечение.')

common_ids = sorted(list(ids1 & ids2))
print('Будем работать с', len(common_ids), 'общими id')

sub1_common = sub1[sub1[ID_COL].isin(common_ids)].copy()
sub2_common = sub2[sub2[ID_COL].isin(common_ids)].copy()

# сортируем по id для аккуратного merge
sub1_common.sort_values(ID_COL, inplace=True)
sub2_common.sort_values(ID_COL, inplace=True)

blend_df = sub1_common[[ID_COL]].copy()
blend_df['answer_1'] = sub1_common[ANSWER_COL].astype(str).fillna('')
blend_df['answer_2'] = sub2_common[ANSWER_COL].astype(str).fillna('')

if REFS_COL in sub1_common.columns and REFS_COL in sub2_common.columns:
    blend_df['refs_1'] = sub1_common[REFS_COL].astype(str).fillna('')
    blend_df['refs_2'] = sub2_common[REFS_COL].astype(str).fillna('')
else:
    blend_df['refs_1'] = ''
    blend_df['refs_2'] = ''

print('\nblend_df preview:')
blend_df.head()


## Block 2. Вспомогательные функции (валидность, длина, похожесть)

In [ ]:
def is_invalid_answer(text: str) -> bool:
    """Эвристика: пустой текст, nan, явные ошибки."""
    if text is None:
        return True
    t = str(text).strip()
    if not t:
        return True
    # часто при RAG встречается что-то вроде 'error: ...'
    lowered = t.lower()
    if lowered.startswith('error:') or 'traceback' in lowered:
        return True
    return False

def answer_length(text: str) -> int:
    if text is None:
        return 0
    return len(str(text))

def simple_similarity(a: str, b: str) -> float:
    """Приблизительная похожесть на [0,1] по SequenceMatcher."""
    a = (a or '').strip()
    b = (b or '').strip()
    if not a and not b:
        return 1.0
    if not a or not b:
        return 0.0
    return difflib.SequenceMatcher(None, a, b).ratio()

def tokenize(text: str) -> List[str]:
    if text is None:
        return []
    return re.findall(r"\w+", str(text).lower(), flags=re.UNICODE)

def jaccard_tokens(a: str, b: str) -> float:
    ta = set(tokenize(a))
    tb = set(tokenize(b))
    if not ta and not tb:
        return 1.0
    if not ta or not tb:
        return 0.0
    inter = len(ta & tb)
    union = len(ta | tb)
    if union == 0:
        return 0.0
    return inter / union


## Block 3. Правила blending: как выбирать лучший ответ

Базовая идея: для каждого `id` у нас есть `answer_1` и `answer_2`.

Мы хотим вернуть `final_answer` и желательно отметить, откуда он взят (`src = 1/2/mix`).

Эвристики:

1. Если один ответ явно **невалидный** (пустой / `error:`), а второй — нет → берём валидный.
2. Если один ответ **очень короткий**, а другой длиннее `MIN_ANSWER_LEN` → берём длинный.
3. Если один ответ **подозрительно длинный** (> `MAX_ANSWER_LEN`), а другой короче → берём короткий.
4. Если ответы **очень похожи** (`similarity > VERY_SIMILAR_THRESHOLD`) → берём более краткий (или от первого сабмита).
5. Иначе по умолчанию доверяем, например, `sub2` (считаем его более сильным) — *или наоборот*, это можно настроить.

In [ ]:
def blend_two_answers(a1: str, a2: str) -> Dict[str, Any]:
    a1 = str(a1 or '')
    a2 = str(a2 or '')

    invalid_1 = is_invalid_answer(a1)
    invalid_2 = is_invalid_answer(a2)

    len1 = answer_length(a1)
    len2 = answer_length(a2)

    sim_seq = simple_similarity(a1, a2)
    sim_jac = jaccard_tokens(a1, a2)

    # 1) если один ответ явно невалидный → берём другой
    if invalid_1 and not invalid_2:
        return {'answer': a2, 'src': 2, 'sim_seq': sim_seq, 'sim_jac': sim_jac}
    if invalid_2 and not invalid_1:
        return {'answer': a1, 'src': 1, 'sim_seq': sim_seq, 'sim_jac': sim_jac}

    # 2) если оба невалидные → возвращаем первый как есть
    if invalid_1 and invalid_2:
        return {'answer': a1, 'src': 1, 'sim_seq': sim_seq, 'sim_jac': sim_jac}

    # 3) короткие ответы vs длинные
    if len1 < MIN_ANSWER_LEN <= len2:
        return {'answer': a2, 'src': 2, 'sim_seq': sim_seq, 'sim_jac': sim_jac}
    if len2 < MIN_ANSWER_LEN <= len1:
        return {'answer': a1, 'src': 1, 'sim_seq': sim_seq, 'sim_jac': sim_jac}

    # 4) слишком длинные ответы
    if len1 > MAX_ANSWER_LEN and len2 <= MAX_ANSWER_LEN:
        return {'answer': a2, 'src': 2, 'sim_seq': sim_seq, 'sim_jac': sim_jac}
    if len2 > MAX_ANSWER_LEN and len1 <= MAX_ANSWER_LEN:
        return {'answer': a1, 'src': 1, 'sim_seq': sim_seq, 'sim_jac': sim_jac}

    # 5) ответы почти одинаковые → берём более короткий
    if sim_seq >= VERY_SIMILAR_THRESHOLD or sim_jac >= VERY_SIMILAR_THRESHOLD:
        if len1 <= len2:
            return {'answer': a1, 'src': 1, 'sim_seq': sim_seq, 'sim_jac': sim_jac}
        else:
            return {'answer': a2, 'src': 2, 'sim_seq': sim_seq, 'sim_jac': sim_jac}

    # 6) дефолт: доверяем sub2 как более "сильному" (это можно поменять)
    return {'answer': a2, 'src': 2, 'sim_seq': sim_seq, 'sim_jac': sim_jac}


## Block 4. Применяем blending ко всем id

In [ ]:
blend_results = []

for _, row in tqdm(blend_df.iterrows(), total=len(blend_df), desc='Blending'):
    a1 = row['answer_1']
    a2 = row['answer_2']
    res = blend_two_answers(a1, a2)
    blend_results.append(res)

blend_meta = pd.DataFrame(blend_results)
blend_meta.head()


In [ ]:
final_df = blend_df[[ID_COL]].copy()
final_df[ANSWER_COL] = blend_meta['answer']
final_df['src'] = blend_meta['src']
final_df['sim_seq'] = blend_meta['sim_seq']
final_df['sim_jac'] = blend_meta['sim_jac']

# Если есть refs_json в обоих сабмитах — берём рефы из того сабмита, чей ответ выбран
final_refs = []
for i, row in final_df.iterrows():
    src = row['src']
    if src == 1:
        final_refs.append(blend_df.loc[i, 'refs_1'])
    elif src == 2:
        final_refs.append(blend_df.loc[i, 'refs_2'])
    else:
        final_refs.append('')

final_df[REFS_COL] = final_refs

print('final_df shape:', final_df.shape)
final_df.head()


## Block 5. (Опционально) Оффлайн-оценка по ground truth

Если у тебя есть файл с правильными ответами `ground_truth.csv` такого формата:

- `id` — вопрос id;
- `answer` — правильный ответ (или эталонный текст),

можно посчитать простую метрику (token-level F1) для:

- сабмита 1;
- сабмита 2;
- blended-сабмита.

Это удобно, чтобы быстро проверить, выгоден ли blending на локальных данных.

In [ ]:
def simple_f1(pred: str, gt: str) -> float:
    pred_tokens = tokenize(pred)
    gt_tokens = tokenize(gt)
    if not pred_tokens or not gt_tokens:
        return 0.0
    common = set(pred_tokens) & set(gt_tokens)
    if not common:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(gt_tokens)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

if GT_PATH.exists():
    gt_df = pd.read_csv(GT_PATH)
    if ID_COL not in gt_df.columns or ANSWER_COL not in gt_df.columns:
        raise KeyError(f'В ground truth должны быть колонки {ID_COL!r} и {ANSWER_COL!r}')

    # совмещаем с финальными сабмитами
    eval_df = gt_df[[ID_COL, ANSWER_COL]].rename(columns={ANSWER_COL: 'gt_answer'})
    eval_df = eval_df.merge(sub1[[ID_COL, ANSWER_COL]].rename(columns={ANSWER_COL: 'pred_1'}), on=ID_COL, how='left')
    eval_df = eval_df.merge(sub2[[ID_COL, ANSWER_COL]].rename(columns={ANSWER_COL: 'pred_2'}), on=ID_COL, how='left')
    eval_df = eval_df.merge(final_df[[ID_COL, ANSWER_COL]].rename(columns={ANSWER_COL: 'pred_blend'}), on=ID_COL, how='left')

    f1_1 = []
    f1_2 = []
    f1_b = []
    for _, r in eval_df.iterrows():
        gt = str(r['gt_answer'])
        f1_1.append(simple_f1(str(r['pred_1']), gt))
        f1_2.append(simple_f1(str(r['pred_2']), gt))
        f1_b.append(simple_f1(str(r['pred_blend']), gt))

    print('\n=== Оффлайн-оценка по token-F1 ===')
    print('sub1   mean F1:', float(np.mean(f1_1)))
    print('sub2   mean F1:', float(np.mean(f1_2)))
    print('blend  mean F1:', float(np.mean(f1_b)))
else:
    print('GT_PATH не существует, оффлайн-оценка пропущена:', GT_PATH)


## Block 6. Сохранение blended-сабмита

In [ ]:
BLEND_PATH = OUTPUT_DIR / 'submission_blended.csv'

# Для сабмита обычно нужны только id и answer (и, возможно, refs_json)
cols_for_submit = [ID_COL, ANSWER_COL]
if REFS_COL in final_df.columns:
    cols_for_submit.append(REFS_COL)

submit_blend = final_df[cols_for_submit].copy()
submit_blend.to_csv(BLEND_PATH, index=False)
print('Blended submission saved to:', BLEND_PATH)
submit_blend.head()


## Итого

В этом ноутбуке:

1. Мы загрузили два сабмита `sub1` и `sub2` и выровняли их по `id`.
2. Для каждого `id` посчитали длину ответов и два типа похожести (SequenceMatcher и Jaccard по токенам).
3. С помощью простых правил выбрали финальный ответ `final_df[answer]` и источник (`src = 1/2`).
4. (Опционально) Посмотрели, улучшает ли blending F1 на локальном ground truth.
5. Сохранили итоговый файл `submission_blended.csv`.

На финале ты можешь быстро подстроить пороги `MIN_ANSWER_LEN`, `MAX_ANSWER_LEN`,
`VERY_SIMILAR_THRESHOLD` и стратегию по умолчанию (кому доверять больше — `sub1` или `sub2`).